# Task Scheduling Analysis

This notebook analyzes task scheduling using various parameters. The code is divided into different sections for better readability and understanding.

In [1]:
import json
import math
from tabulate import tabulate

FILE_PATH = 'input.json'

## Task Class

The `Task` class represents a single task with its id, execution time, deadline, period, and utilization. It also includes methods to calculate the utilization and generate deadlines up to a maximum possible deadline.

In [2]:
class Task:
    """Represents a single task with its attributes and methods."""
    
    def __init__(self, task_id, exec_time, deadline, period):
        """
        Initializes a Task instance.
        
        Args:
            task_id (str): Unique identifier for the task.
            exec_time (int): Execution time of the task.
            deadline (int): Deadline of the task.
            period (int): Period of the task.
        """
        self.task_id = task_id
        self.exec_time = exec_time
        self.deadline = deadline
        self.period = period
        self.utilization = exec_time / period

    def get_utilization(self):
        """
        Returns the utilization of the task.
        
        Returns:
            float: Utilization of the task.
        """
        return self.utilization

    def get_deadlines(self, max_possible_deadline):
        """
        Generates a list of deadlines up to the maximum possible deadline.
        
        Args:
            max_possible_deadline (int): The maximum possible deadline.
        
        Returns:
            list: List of deadlines.
        """
        deadlines = []
        deadline = self.deadline
        while deadline < max_possible_deadline:
            deadlines.append(deadline)
            deadline += self.period
        return deadlines

## TaskScheduler Class

The `TaskScheduler` class handles a collection of tasks. It includes methods to calculate total utilization, critical time (L*), hyper period, deadlines, and the demand bound function (g). It also runs the reducing test intervals to determine if the tasks can be scheduled.

In [3]:
class TaskScheduler:
    """Handles a collection of tasks and performs scheduling analysis."""
    
    def __init__(self, tasks):
        """
        Initializes the TaskScheduler with a list of tasks.
        
        Args:
            tasks (list): List of task dictionaries.
        """
        self.tasks = [
            Task(task_id=task['id'],
            exec_time=task['execution_time'],
            deadline=task['deadline'],
            period=task['period'])
            for task in tasks
        ]

    def total_utilization(self):
        """
        Calculates the total utilization of all tasks.
        
        Returns:
            float: Total utilization of all tasks.
        """
        return sum(task.get_utilization() for task in self.tasks)

    def compute_l_star(self):
        """
        Computes the critical time L* for the tasks.
        
        Returns:
            int: The critical time L*.
        """
        l_star = sum((task.period - task.deadline) * task.utilization for task in self.tasks)
        return math.ceil(l_star / (1 - self.total_utilization()))

    def compute_hyper_period(self):
        """
        Computes the hyper period (H) of the tasks.
        
        Returns:
            int: The hyper period.
        """
        periods = [task.period for task in self.tasks]
        return math.lcm(*periods)

    def compute_deadlines(self):
        """
        Computes a list of unique deadlines for the tasks.
        
        Returns:
            list: List of unique deadlines.
        """
        max_deadline = max(task.deadline for task in self.tasks)
        hyper_period = self.compute_hyper_period()
        l_star = self.compute_l_star()
        max_possible_deadline = min(hyper_period, max(max_deadline, l_star))

        deadlines = []
        for task in self.tasks:
            deadlines.extend(task.get_deadlines(max_possible_deadline))
        return list(set(deadlines))

    def compute_g(self, l):
        """
        Computes the demand bound function g at time L.
        
        Args:
            l (int): The time L.
        
        Returns:
            int: The demand bound function g at time L.
        """
        return sum(math.floor((l + task.period - task.deadline) / task.period) * task.exec_time for task in self.tasks)

    def run(self):
        """
        Runs the reducing test intervals and generates a result table.
        
        Returns:
            list: Result table of test intervals with L, g(0, L), and result.
        """
        result_table = []
        deadlines = sorted(self.compute_deadlines())
        for l in deadlines:
            g_l = self.compute_g(l)
            result = 'OK' if g_l <= l else 'NOT OK'
            result_table.append([l, g_l, result])
            if result == 'NOT OK':
                break
        return result_table

## Main Function

The `main` function reads the task data from a JSON file, initializes the `TaskScheduler` with this data, and then calculates and prints the various scheduling parameters and results.

In [4]:
def main():
    """
    Main function to read task data, initialize scheduler, and print results.
    """
    with open(FILE_PATH) as f:
        tasks_data = json.load(f)
    scheduler = TaskScheduler(tasks_data)


    utilization = scheduler.total_utilization()
    l_star = scheduler.compute_l_star()
    hyper_period = scheduler.compute_hyper_period()
    deadlines = scheduler.compute_deadlines()
    test_intervals = scheduler.run()

    print("Summary of Results")
    print("------------------")
    print(f"Total Utilization (U) = {utilization:.2f}")
    print(f"Critical Time (L*) = {l_star}")
    print(f"Hyper Period (H) = {hyper_period}")
    print(f"Deadlines (D) = {', '.join(map(str, deadlines))}\n")

    print("Detailed Analysis")
    print("-----------------")
    print(tabulate(test_intervals, headers=['L', 'g(0, L)', 'Result'], tablefmt="fancy_grid"))


if __name__ == '__main__':
    main()

Summary of Results
------------------
Total Utilization (U) = 0.90
Critical Time (L*) = 35
Hyper Period (H) = 60
Deadlines (D) = 8, 12, 15, 18, 27, 28

Detailed Analysis
-----------------
╒═════╤═══════════╤══════════╕
│   L │   g(0, L) │ Result   │
╞═════╪═══════════╪══════════╡
│   8 │         1 │ OK       │
├─────┼───────────┼──────────┤
│  12 │         7 │ OK       │
├─────┼───────────┼──────────┤
│  15 │        15 │ OK       │
├─────┼───────────┼──────────┤
│  18 │        16 │ OK       │
├─────┼───────────┼──────────┤
│  27 │        22 │ OK       │
├─────┼───────────┼──────────┤
│  28 │        23 │ OK       │
╘═════╧═══════════╧══════════╛


### Description of Main Function

- **Read Task Data**: Reads the task data from the JSON file specified by `FILE_PATH`.
- **Initialize TaskScheduler**: Initializes the `TaskScheduler` with the loaded tasks.
- **Calculate Utilization**: Calculates the total utilization of the tasks.
- **Calculate Critical Time**: Computes the critical time `L*`.
- **Calculate Hyper Period**: Determines the hyper period `H`.
- **Calculate Deadlines**: Computes the list of deadlines.
- **Run Reducing Test Intervals**: Runs the reducing test intervals to check if the tasks can be scheduled.
- **Print Summary and Detailed Results**: Prints a summary of the results followed by a detailed analysis in a tabulated format.